In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import warnings


In [3]:
rcParams['figure.figsize'] = (16,8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns',100)
pd.set_option("display.precision",4)
warnings.simplefilter('ignore')

# 학습 데이터 로드 . 
기존에 사용했었던 feature.csv 를 사용할 것이다. 

In [4]:
data_dir = Path('./Dacon')
feature_dir = Path ('./Dacon')
sub_dir = Path ('./decision_tree')

tnr_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'class'
seed = 42

In [5]:
algo_name = 'rf'
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
sub_file = sub_dir / f'{model_name}.csv'

In [7]:
df = pd.read_csv (feature_file, index_col=0)
df.head()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
id,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,13,12,1.1939,1.1943,1.1937,1.1938,1.1941,0


In [8]:
y = df[target_col].values[ :320000]
df.drop(target_col, axis=1, inplace = True)
trn = df.iloc[:320000]
tst = df.iloc[320000:]
feature_name = df.columns.tolist()
print(y.shape,trn.shape,tst.shape)

(319820,) (319820, 18) (0, 18)


In [25]:
trn

,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,airmass_u,d_dered_u,d_dered_g,d_dered_r,d_dered_i,d_dered_z,d_dered_ig,d_dered_zg,d_dered_rz,d_dered_iz,d_obs_det
id,,,,,,,,,,,,,,,,,,,
0,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,2.9444,1.1898,-0.1397,-0.0790,-0.0544,-0.0403,-0.0307,-2.6257,-3.3488,2.0462,0.7232,-15.0556
1,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,0.6931,1.2533,-0.0857,-0.0574,-0.0410,-0.0322,-0.0343,-0.7683,-0.8698,0.2767,0.1016,-0.3069
2,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,1.0986,1.0225,-0.1787,-0.1388,-0.0963,-0.0718,-0.0540,-0.3649,-0.3905,0.1440,0.0257,-0.9014
3,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,1.6094,1.2054,-0.3070,-0.1941,-0.1339,-0.1003,-0.0795,-1.2000,-1.4395,0.5321,0.2395,-1.3906
4,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,2.6391,1.1939,-0.6820,-0.2653,-0.1794,-0.1339,-0.1067,-2.2436,-2.5579,0.9871,0.3144,-9.3609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319995,16.3404,5.1988e-02,18.1328,17.0406,16.6374,16.3861,16.2874,1.0986,1.0930,-0.2328,-0.2002,-0.1365,-0.0989,-0.0530,-0.6546,-0.7532,0.3500,0.0986,-0.9014
319996,16.6367,8.7261e-02,18.5207,17.5131,17.0871,16.7436,16.6017,1.0986,1.0517,-0.1649,-0.1527,-0.1033,-0.0732,-0.0350,-0.7695,-0.9114,0.4854,0.1419,-0.9014
319997,16.6841,-1.0148e-05,20.9890,18.3076,17.2152,16.8403,16.6355,3.4657,1.1933,-0.2000,-0.1158,-0.0804,-0.0608,-0.0486,-1.4673,-1.6721,0.5796,0.2048,-26.5343


In [26]:
tst

,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,airmass_u,d_dered_u,d_dered_g,d_dered_r,d_dered_i,d_dered_z,d_dered_ig,d_dered_zg,d_dered_rz,d_dered_iz,d_obs_det
id,,,,,,,,,,,,,,,,,,,
320000,16.9263,1.2130e-01,19.0750,17.9550,17.4308,17.0838,16.9087,0.6931,1.1563,-0.0681,-0.0622,-0.0424,-0.0306,-0.0176,-0.8711,-1.0462,0.5221,0.1751,-0.3069
320001,15.7307,-6.6907e-05,15.0703,14.3098,14.3901,14.5160,14.6372,1.0986,1.0076,-3.6706,-2.8607,-1.9790,-1.4705,-1.0935,0.2062,0.3274,-0.2470,-0.1212,-0.9014
320002,16.6352,1.1001e-01,19.1004,17.7726,17.1460,16.7497,16.5689,0.6931,1.3809,-0.1977,-0.1077,-0.0761,-0.0599,-0.0664,-1.0230,-1.2037,0.5771,0.1808,-0.3069
320003,16.7955,2.9697e-05,17.9729,17.0735,16.8470,16.7700,16.7668,0.6931,1.1408,-0.1045,-0.0834,-0.0572,-0.0418,-0.0288,-0.3036,-0.3068,0.0802,0.0032,-0.3069
320004,15.6621,6.8471e-02,17.7545,16.6794,16.1563,15.8315,15.6433,1.0986,1.1110,-0.0780,-0.0852,-0.0572,-0.0407,-0.0187,-0.8480,-1.0361,0.5129,0.1881,-0.9014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,18.0627,-3.1726e-04,19.3361,18.3933,18.0860,17.9861,17.9670,2.8904,1.1858,-0.3372,-0.2684,-0.1848,-0.1364,-0.0957,-0.4072,-0.4263,0.1190,0.0191,-14.1096
399996,20.8458,6.5455e-02,21.3993,20.6399,20.5451,20.9507,20.7827,2.1972,1.2850,-0.1235,-0.0884,-0.0636,-0.0542,-0.0631,0.3108,0.1428,-0.2376,0.1680,-1.8028
399997,18.2018,-8.4180e-04,18.9136,17.8753,17.9047,18.0043,18.0400,0.6931,1.1885,-0.4855,-0.3657,-0.2558,-0.1935,-0.1618,0.1289,0.1647,-0.1353,-0.0357,-0.3069


# 학습/ 검증 데이터 구분

# train_test_split 함수 사용하는 방법. 

trn, y :학습 데이터와 종속 변수를 넣은 상황. 

test_size : 학습할 테스트 데이터 셋의 비율

random state : 데이터 분할 시 셔플이 이루어 지는데 이를 위한 시드값. 


In [30]:
X_trn, X_val, y_trn, y_val = train_test_split(trn, y, test_size=.2, random_state=seed)
print(X_trn.shape, X_val.shape, y_trn.shape, y_val.shape)


(256000, 19) (64000, 19) (256000,) (64000,)


# Random forest 모델 학습 시작. 

In [34]:
clf = RandomForestClassifier(n_estimators = 100,
                             min_samples_leaf=10,
                            max_features='auto',
                            max_samples=.5,
                            random_state=seed,
                            n_jobs=-1)
clf.fit(X_trn,y_trn)
p_val = clf.predict(X_val)
p_tst = clf.predict(tst)

In [35]:
print(f'{accuracy_score(y_val, p_val) * 100:.4f}%')

92.4531%
